In [6]:
from operator import itemgetter
a = [[1,2,3], [0,1,2], [1,3,6],[12,36,4]]
sorted_list = sorted(a, key=itemgetter(0))
print sorted_list

[[0, 1, 2], [1, 2, 3], [1, 3, 6], [12, 36, 4]]


In [358]:
import xml.etree.ElementTree
import csv
import matplotlib.pyplot as plt
from random import shuffle
%matplotlib inline

In [359]:
%run utils/utilities.py
%run HAR-CNN-GPU.py

ERROR:root:File `u'HAR-CNN-GPU.py'` not found.


In [360]:
class_label_vn = [u"Cổ tay",u"Cổ chân",u"Bả vai",u"Xoay người",u"Xoay đầu gối",u"Đi bộ",u"Chạy",u"Đá bóng",u"Đạp xe",u"Đánh răng",u"Rửa tay", u"Lau bàn", u"Quét nhà", u"Nạo",u"Thái",u"Trộn",u"Lên cầu thang",u"Xuống cầu thang"]
# class_label=["wrist","ankle","shoulder","haunch","knee","walking","running","kicking","cycling","brushing","washing_hand","wipe","sweep_floor","peel","slice","mixing","upstair","downstair"]

In [361]:
DATASET_ROOT = './datasets/PTIT'
DATASET_NORM = DATASET_ROOT + '/normalized'
DATASET_TRAIN = DATASET_NORM + '/train'
DATASET_TEST = DATASET_NORM + '/test'
WINDONW_OVERLAP = 0.5
WINDOWN_OVERLAP_SIZE = WINDONW_OVERLAP * WINDOWN_SIZE
ANNO_FILE = 'anno.eaf'
WAX3_FILE = 'wax3.csv'
GEARS2_FILE = 'gears2.csv'

In [362]:
train_window_cnt = {}
test_window_cnt = {}
total_window = 0

In [363]:
def getAnno(anno_path):
    timestamp = {}
    annotation = []
    time_range = []
    root_ele = xml.etree.ElementTree.parse(anno_path).getroot()
    for time_slot in root_ele.iter('TIME_SLOT'):
        att = time_slot.attrib
        timestamp[att['TIME_SLOT_ID']]  = att['TIME_VALUE']
    for anno in root_ele.find('TIER').iter('ANNOTATION'):
        alig_anno = anno.find('ALIGNABLE_ANNOTATION')
        anno_text = alig_anno.find('ANNOTATION_VALUE').text.strip()
        if anno_text not in class_label:
            continue
        startTs = timestamp[alig_anno.attrib['TIME_SLOT_REF1']]
        endTs = timestamp[alig_anno.attrib['TIME_SLOT_REF2']]
#         annotation[anno_text] = {'start': startTs, 'end': endTs}
        annotation.append(anno_text)
        time_range.append({'start': int(startTs), 'end': int(endTs)})
    return annotation, time_range

def getMilisecond(s):
    try:
        hours, minutes, seconds = (["0", "0"] + s.split(":"))[-3:]
        hours = int(hours)
        minutes = int(minutes)
        seconds = float(seconds)
        miliseconds = int(3600000 * hours + 60000 * minutes + 1000 * seconds)
        return miliseconds
    except:
#         print "format exception " + s
        return 0

def getTotalWindowSize(len):
    total_size = (len // WINDOWN_SIZE) * 2 - 1 + (len % WINDOWN_SIZE) // WINDOWN_OVERLAP_SIZE
    return int(total_size)

In [364]:
def exportData(dir_path, target_path, is_training=True):
    global total_window
    anno_file = dir_path + '/' + ANNO_FILE
    sensor_data_path = target_path + '/sensor'
    x_watch_acc_file = sensor_data_path + '/x_watch_acc.txt'
    y_watch_acc_file = sensor_data_path + '/y_watch_acc.txt'
    z_watch_acc_file = sensor_data_path + '/z_watch_acc.txt'
    x_watch_gyr_file = sensor_data_path + '/x_watch_gyr.txt'
    y_watch_gyr_file = sensor_data_path + '/y_watch_gyr.txt'
    z_watch_gyr_file = sensor_data_path + '/z_watch_gyr.txt'
    x_sensor_acc_file = sensor_data_path + '/x_sensor_acc.txt'
    y_sensor_acc_file = sensor_data_path + '/y_sensor_acc.txt'
    z_sensor_acc_file = sensor_data_path + '/z_sensor_acc.txt'
    class_file = target_path + '/class.txt'
    annotation, time_range = getAnno(anno_file)
    num_anno = len(annotation)
    gears2_data = [[] for anno in annotation];
    wax3_data = [[] for anno in annotation];
    start_annotation = 0
    wax3_file = dir_path + '/' + WAX3_FILE
    gears2_file = dir_path + '/' + GEARS2_FILE
    with open(gears2_file, 'r') as gears2_csv, open(wax3_file, 'r') as wax3_csv:
        gears2_csv_reader = csv.reader(gears2_csv, delimiter=',')
        wax3_csv_reader = csv.reader(wax3_csv, delimiter=',')
        for row in gears2_csv_reader:
            ts = getMilisecond(row[0].strip())
            for i in range(num_anno):
                if ts >= time_range[i]['start'] and ts < time_range[i]['end']:
                    x_acc = float(row[1].strip())
                    y_acc = float(row[2].strip())
                    z_acc = float(row[3].strip())
                    x_gyr = float(row[4].strip())
                    y_gyr = float(row[5].strip())
                    z_gyr = float(row[6].strip())
                    gears2_data[i].append([ts, x_acc, y_acc, z_acc, x_gyr, y_gyr, z_gyr])
                    
        for row in wax3_csv_reader:
            ts = getMilisecond(row[0].strip())
            for i in range(num_anno):
                if ts >= time_range[i]['start'] and ts < time_range[i]['end']:
                    x_acc = float(row[1].strip())
                    y_acc = float(row[2].strip())
                    z_acc = float(row[3].strip())
                    wax3_data[i].append([ts, x_acc, y_acc, z_acc])
         
    for i in range(num_anno):
        num_windows = getTotalWindowSize(len(gears2_data[i]))
        startWindow = 0
#         print num_windows
        for j in range(num_windows):
            windowSliced = gears2_data[i][startWindow:startWindow+WINDOWN_SIZE]
            if(len(windowSliced) < WINDOWN_SIZE):
                break
            startWindow += WINDOWN_SIZE
            startTs = windowSliced[0][0]
            endTs = windowSliced[WINDOWN_SIZE - 1][0]
            equivWax3Data = []
            # get data from wax3 and group with gears2
            for k in range(len(wax3_data[i])):
                wax3Ts = wax3_data[i][k][0]
                if(wax3Ts > endTs):
                    break
                if wax3Ts >= startTs and wax3Ts <= endTs:
                    equivWax3Data.append(wax3_data[i][k])
            zero_arr = [0 for zit in range(WINDOWN_SIZE)]
            while len(equivWax3Data) < WINDOWN_SIZE:
                equivWax3Data.append(zero_arr);
            equivWax3Data = equivWax3Data[0:WINDOWN_SIZE]
#             print (startTs, endTs, len(windowSliced), len(equivWax3Data))
            #export windows to file
            x_watch_acc = y_watch_acc = z_watch_acc = "";
            x_watch_gyr = y_watch_gyr = z_watch_gyr = "";
            x_sensor_acc = y_sensor_acc = z_sensor_acc = "";
#             if len(windowSliced) > 150 or len(equivWax3Data) > 150:
#                 print(i, j, len(windowSliced), len(equivWax3Data))
            for k in range(WINDOWN_SIZE):
                x_watch_acc = x_watch_acc + " " + str(windowSliced[k][1])
                y_watch_acc = y_watch_acc + " " + str(windowSliced[k][2])
                z_watch_acc = z_watch_acc + " " + str(windowSliced[k][3])
                
                x_watch_gyr = x_watch_gyr + " " + str(windowSliced[k][4])
                y_watch_gyr = y_watch_gyr + " " + str(windowSliced[k][5])
                z_watch_gyr = z_watch_gyr + " " + str(windowSliced[k][6])
                
                x_sensor_acc = x_sensor_acc + " " + str(equivWax3Data[k][1])
                y_sensor_acc = y_sensor_acc + " " + str(equivWax3Data[k][2])
                z_sensor_acc = z_sensor_acc + " " + str(equivWax3Data[k][3])
                

            with open(x_watch_acc_file, "a") as fw:
                fw.write(x_watch_acc + "\n")
            with open(y_watch_acc_file, "a") as fw:
                fw.write(y_watch_acc + "\n")
            with open(z_watch_acc_file, "a") as fw:
                fw.write(z_watch_acc + "\n")
                
            with open(x_watch_gyr_file, "a") as fw:
                fw.write(x_watch_gyr + "\n")
            with open(y_watch_gyr_file, "a") as fw:
                fw.write(y_watch_gyr + "\n")
            with open(z_watch_gyr_file, "a") as fw:
                fw.write(z_watch_gyr + "\n")
                
            with open(x_sensor_acc_file, "a") as fw:
                fw.write(x_sensor_acc + "\n")
            with open(y_sensor_acc_file, "a") as fw:
                fw.write(y_sensor_acc + "\n")
            with open(z_sensor_acc_file, "a") as fw:
                fw.write(z_sensor_acc + "\n")
            with open(class_file, "a") as fw:
                fw.write(str(class_label_int[annotation[i]]) + "\n")
        train_window_cnt[class_label_vn[int(class_label_int[annotation[i]])]]+=num_windows
        total_window+=num_windows;
#     return wax3_data
        
# exportData('./datasets/PTIT/001/in/', DATASET_TRAIN)

In [365]:
def prepareTrainTestFile(trainDir, testDir, name="default"):
    for dirPath in trainDir:
#         print dirPath
        exportData(DATASET_ROOT + '/' + dirPath + '/in', DATASET_TRAIN, True)
        exportData(DATASET_ROOT + '/' + dirPath + '/out', DATASET_TRAIN, True)
    for dirPath in testDir:
#         print dir
        exportData(DATASET_ROOT + '/' + dirPath + '/in', DATASET_TEST, False)
        exportData(DATASET_ROOT + '/' + dirPath + '/out', DATASET_TEST, False)

In [366]:
def prepareTempTrainTestFormat(test_dir):
    allDir = ["001", "002", "003", "004", "005", "006", "007", "008", "009", "010", "011","013", "014"]
    trainDir = []
    for cdir in allDir:
        if cdir not in test_dir:
            trainDir.append(cdir)
    global train_window_cnt
    global test_window_cnt
    global total_train_window
    total_train_window = 0
    train_window_cnt = dict.fromkeys(class_label_vn, 0)
    test_window_cnt = dict.fromkeys(class_label_vn, 0)
    prepareTrainTestFile(trainDir, test_dir)
    print ("training window count ", train_window_cnt)
    print ("test window count ", test_window_cnt)

In [367]:
def randomTrainFile():
    records = None
    random_dir = DATASET_NORM + "_random/train"
    files = [DATASET_TRAIN + "/" + "class.txt"]
    files_random  = [random_dir + "/" + "class.txt"]
    for channel in CHANNEL_LIST:
        filePath = DATASET_TRAIN + "/sensor/" + channel + ".txt"
        files.append(filePath)
        files_random.append(random_dir + "/sensor/" + channel + ".txt")
    for file in files:
        print "read file " + file
        with open(file) as f:
            lines = f.readlines()
            lines = [line.strip() for line in lines]
            if records == None:
#                 print "create records"
                records = [[] for _ in range(len(lines))]
            for idx, line in enumerate(lines):
                records[idx].append(line)
#                 print (idx, line, records[idx][0])
#             return
#     print "record len %d" % len(records)
    shuffle(records)
    total10 = total0 = 0;
    for file_idx, file in enumerate(files_random):
#         print "write file " + file
        with open(file, "a") as wf:
            for record in records:
#                 print record[file_idx]
                wf.write(record[file_idx] + "\n")
    print(total0, total10)

In [368]:
!rm -rf datasets/PTIT/normalized*
#!mkdir -p datasets/PTIT/normalized/{train,test}/sensor
!mkdir -p datasets/PTIT/normalized/train/sensor
!mkdir -p datasets/PTIT/normalized/test/sensor
!mkdir -p datasets/PTIT/normalized_random/train/sensor
!mkdir -p datasets/PTIT/normalized_random/test/sensor
prepareTempTrainTestFormat(["014"])
# randomTrainFile()


In [369]:
for cdir in ["001"]:
    !rm -rf datasets/PTIT/normalized*
    #!mkdir -p datasets/PTIT/normalized/{train,test}/sensor
    !mkdir -p datasets/PTIT/normalized/train/sensor
    !mkdir -p datasets/PTIT/normalized/test/sensor
    !mkdir -p datasets/PTIT/normalized_random/train/sensor
    !mkdir -p datasets/PTIT/normalized_random/test/sensor
    prepareTempTrainTestFormat([cdir])
    randomTrainFile()
    train()

('training window count ', {u'Lau b\xe0n': 586, u'Ch\u1ea1y': 922, u'L\xean c\u1ea7u thang': 585, u'\u0110\u1ea1p xe': 921, u'Qu\xe9t nh\xe0': 655, u'Xu\u1ed1ng c\u1ea7u thang': 490, u'N\u1ea1o': 942, u'\u0110\xe1nh r\u0103ng': 507, u'Tr\u1ed9n': 912, u'C\u1ed5 tay': 458, u'\u0110i b\u1ed9': 988, u'\u0110\xe1 b\xf3ng': 1034, u'R\u1eeda tay': 602, u'Xoay ng\u01b0\u1eddi': 459, u'B\u1ea3 vai': 426, u'Th\xe1i': 955, u'C\u1ed5 ch\xe2n': 499, u'Xoay \u0111\u1ea7u g\u1ed1i': 443})
('test window count ', {u'Lau b\xe0n': 0, u'Ch\u1ea1y': 0, u'L\xean c\u1ea7u thang': 0, u'\u0110\u1ea1p xe': 0, u'Qu\xe9t nh\xe0': 0, u'Xu\u1ed1ng c\u1ea7u thang': 0, u'N\u1ea1o': 0, u'\u0110\xe1nh r\u0103ng': 0, u'Tr\u1ed9n': 0, u'C\u1ed5 tay': 0, u'\u0110i b\u1ed9': 0, u'\u0110\xe1 b\xf3ng': 0, u'R\u1eeda tay': 0, u'Xoay ng\u01b0\u1eddi': 0, u'B\u1ea3 vai': 0, u'Th\xe1i': 0, u'C\u1ed5 ch\xe2n': 0, u'Xoay \u0111\u1ea7u g\u1ed1i': 0})
read file ./datasets/PTIT/normalized/train/class.txt
read file ./datasets/PTIT/no

('Epoch: 43/1000', 'Iteration: 390', 'Train loss: 1.567227', 'Train acc: 0.521667')
('Epoch: 43/1000', 'Iteration: 395', 'Train loss: 1.769557', 'Train acc: 0.423333')
('Epoch: 44/1000', 'Iteration: 400', 'Train loss: 1.651191', 'Train acc: 0.435000')
('Epoch: 44/1000', 'Iteration: 405', 'Train loss: 1.599822', 'Train acc: 0.498333')
('Epoch: 45/1000', 'Iteration: 410', 'Train loss: 1.610986', 'Train acc: 0.540000')
('Epoch: 46/1000', 'Iteration: 415', 'Train loss: 1.651920', 'Train acc: 0.466667')
('Epoch: 46/1000', 'Iteration: 420', 'Train loss: 2.058679', 'Train acc: 0.321667')
('Epoch: 47/1000', 'Iteration: 425', 'Train loss: 1.704773', 'Train acc: 0.398333')
('Epoch: 47/1000', 'Iteration: 430', 'Train loss: 1.528300', 'Train acc: 0.531667')
('Epoch: 48/1000', 'Iteration: 435', 'Train loss: 1.458298', 'Train acc: 0.528333')
('Epoch: 48/1000', 'Iteration: 440', 'Train loss: 1.678253', 'Train acc: 0.488333')
('Epoch: 49/1000', 'Iteration: 445', 'Train loss: 1.528485', 'Train acc: 0.4

('Epoch: 97/1000', 'Iteration: 880', 'Train loss: 0.868814', 'Train acc: 0.718333')
('Epoch: 98/1000', 'Iteration: 885', 'Train loss: 0.971580', 'Train acc: 0.710000')
('Epoch: 98/1000', 'Iteration: 890', 'Train loss: 1.235080', 'Train acc: 0.578333')
('Epoch: 99/1000', 'Iteration: 895', 'Train loss: 1.052807', 'Train acc: 0.600000')
('Epoch: 99/1000', 'Iteration: 900', 'Train loss: 0.788497', 'Train acc: 0.820000')
('Epoch: 100/1000', 'Iteration: 905', 'Train loss: 1.149953', 'Train acc: 0.676667')
('Epoch: 101/1000', 'Iteration: 910', 'Train loss: 0.956520', 'Train acc: 0.713333')
('Epoch: 101/1000', 'Iteration: 915', 'Train loss: 1.412035', 'Train acc: 0.500000')
('Epoch: 102/1000', 'Iteration: 920', 'Train loss: 0.973339', 'Train acc: 0.703333')
('Epoch: 102/1000', 'Iteration: 925', 'Train loss: 0.841457', 'Train acc: 0.718333')
('Epoch: 103/1000', 'Iteration: 930', 'Train loss: 0.915664', 'Train acc: 0.726667')
('Epoch: 103/1000', 'Iteration: 935', 'Train loss: 1.209692', 'Train a

('Epoch: 151/1000', 'Iteration: 1360', 'Train loss: 0.722452', 'Train acc: 0.806667')
('Epoch: 151/1000', 'Iteration: 1365', 'Train loss: 1.102122', 'Train acc: 0.631667')
('Epoch: 152/1000', 'Iteration: 1370', 'Train loss: 0.703496', 'Train acc: 0.771667')
('Epoch: 152/1000', 'Iteration: 1375', 'Train loss: 0.622648', 'Train acc: 0.808333')
('Epoch: 153/1000', 'Iteration: 1380', 'Train loss: 0.737574', 'Train acc: 0.781667')
('Epoch: 153/1000', 'Iteration: 1385', 'Train loss: 1.031171', 'Train acc: 0.680000')
('Epoch: 154/1000', 'Iteration: 1390', 'Train loss: 0.829265', 'Train acc: 0.688333')
('Epoch: 154/1000', 'Iteration: 1395', 'Train loss: 0.592275', 'Train acc: 0.840000')
('Epoch: 155/1000', 'Iteration: 1400', 'Train loss: 0.868898', 'Train acc: 0.750000')
('Epoch: 156/1000', 'Iteration: 1405', 'Train loss: 0.736601', 'Train acc: 0.785000')
('Epoch: 156/1000', 'Iteration: 1410', 'Train loss: 1.091603', 'Train acc: 0.636667')
('Epoch: 157/1000', 'Iteration: 1415', 'Train loss: 0.

('Epoch: 204/1000', 'Iteration: 1840', 'Train loss: 0.737845', 'Train acc: 0.693333')
('Epoch: 204/1000', 'Iteration: 1845', 'Train loss: 0.466240', 'Train acc: 0.886667')
('Epoch: 205/1000', 'Iteration: 1850', 'Train loss: 0.718069', 'Train acc: 0.790000')
('Epoch: 206/1000', 'Iteration: 1855', 'Train loss: 0.590669', 'Train acc: 0.818333')
('Epoch: 206/1000', 'Iteration: 1860', 'Train loss: 0.870895', 'Train acc: 0.728333')
('Epoch: 207/1000', 'Iteration: 1865', 'Train loss: 0.527460', 'Train acc: 0.843333')
('Epoch: 207/1000', 'Iteration: 1870', 'Train loss: 0.492766', 'Train acc: 0.836667')
('Epoch: 208/1000', 'Iteration: 1875', 'Train loss: 0.622087', 'Train acc: 0.813333')
('Epoch: 208/1000', 'Iteration: 1880', 'Train loss: 0.893372', 'Train acc: 0.721667')
('Epoch: 209/1000', 'Iteration: 1885', 'Train loss: 0.689536', 'Train acc: 0.708333')
('Epoch: 209/1000', 'Iteration: 1890', 'Train loss: 0.456658', 'Train acc: 0.881667')
('Epoch: 210/1000', 'Iteration: 1895', 'Train loss: 0.

...

('Epoch: 417/1000', 'Iteration: 3760', 'Train loss: 0.238514', 'Train acc: 0.933333')
('Epoch: 418/1000', 'Iteration: 3765', 'Train loss: 0.370648', 'Train acc: 0.898333')
('Epoch: 418/1000', 'Iteration: 3770', 'Train loss: 0.533168', 'Train acc: 0.845000')
('Epoch: 419/1000', 'Iteration: 3775', 'Train loss: 0.385595', 'Train acc: 0.868333')
('Epoch: 419/1000', 'Iteration: 3780', 'Train loss: 0.231023', 'Train acc: 0.923333')
('Epoch: 420/1000', 'Iteration: 3785', 'Train loss: 0.441036', 'Train acc: 0.880000')
('Epoch: 421/1000', 'Iteration: 3790', 'Train loss: 0.310914', 'Train acc: 0.920000')
('Epoch: 421/1000', 'Iteration: 3795', 'Train loss: 0.397370', 'Train acc: 0.890000')
('Epoch: 422/1000', 'Iteration: 3800', 'Train loss: 0.253801', 'Train acc: 0.926667')
('Epoch: 422/1000', 'Iteration: 3805', 'Train loss: 0.243079', 'Train acc: 0.931667')
('Epoch: 423/1000', 'Iteration: 3810', 'Train loss: 0.353358', 'Train acc: 0.906667')
('Epoch: 423/1000', 'Iteration: 3815', 'Train loss: 0.

('Epoch: 471/1000', 'Iteration: 4240', 'Train loss: 0.272347', 'Train acc: 0.923333')
('Epoch: 471/1000', 'Iteration: 4245', 'Train loss: 0.337707', 'Train acc: 0.920000')
('Epoch: 472/1000', 'Iteration: 4250', 'Train loss: 0.218720', 'Train acc: 0.935000')
('Epoch: 472/1000', 'Iteration: 4255', 'Train loss: 0.235964', 'Train acc: 0.918333')
('Epoch: 473/1000', 'Iteration: 4260', 'Train loss: 0.325678', 'Train acc: 0.903333')
('Epoch: 473/1000', 'Iteration: 4265', 'Train loss: 0.449260', 'Train acc: 0.863333')
('Epoch: 474/1000', 'Iteration: 4270', 'Train loss: 0.320480', 'Train acc: 0.881667')
('Epoch: 474/1000', 'Iteration: 4275', 'Train loss: 0.186696', 'Train acc: 0.941667')
('Epoch: 475/1000', 'Iteration: 4280', 'Train loss: 0.374473', 'Train acc: 0.881667')
('Epoch: 476/1000', 'Iteration: 4285', 'Train loss: 0.273041', 'Train acc: 0.923333')
('Epoch: 476/1000', 'Iteration: 4290', 'Train loss: 0.332896', 'Train acc: 0.903333')
('Epoch: 477/1000', 'Iteration: 4295', 'Train loss: 0.

('Epoch: 524/1000', 'Iteration: 4720', 'Train loss: 0.305277', 'Train acc: 0.900000')
('Epoch: 524/1000', 'Iteration: 4725', 'Train loss: 0.186736', 'Train acc: 0.945000')
('Epoch: 525/1000', 'Iteration: 4730', 'Train loss: 0.363707', 'Train acc: 0.883333')
('Epoch: 526/1000', 'Iteration: 4735', 'Train loss: 0.209392', 'Train acc: 0.948333')
('Epoch: 526/1000', 'Iteration: 4740', 'Train loss: 0.289330', 'Train acc: 0.936667')
('Epoch: 527/1000', 'Iteration: 4745', 'Train loss: 0.205315', 'Train acc: 0.933333')
('Epoch: 527/1000', 'Iteration: 4750', 'Train loss: 0.193032', 'Train acc: 0.941667')
('Epoch: 528/1000', 'Iteration: 4755', 'Train loss: 0.261468', 'Train acc: 0.928333')
('Epoch: 528/1000', 'Iteration: 4760', 'Train loss: 0.403646', 'Train acc: 0.865000')
('Epoch: 529/1000', 'Iteration: 4765', 'Train loss: 0.297438', 'Train acc: 0.886667')
('Epoch: 529/1000', 'Iteration: 4770', 'Train loss: 0.179558', 'Train acc: 0.943333')
('Epoch: 530/1000', 'Iteration: 4775', 'Train loss: 0.

('Epoch: 577/1000', 'Iteration: 5200', 'Train loss: 0.167500', 'Train acc: 0.955000')
('Epoch: 578/1000', 'Iteration: 5205', 'Train loss: 0.252527', 'Train acc: 0.933333')
('Epoch: 578/1000', 'Iteration: 5210', 'Train loss: 0.353475', 'Train acc: 0.893333')
('Epoch: 579/1000', 'Iteration: 5215', 'Train loss: 0.244680', 'Train acc: 0.911667')
('Epoch: 579/1000', 'Iteration: 5220', 'Train loss: 0.143162', 'Train acc: 0.958333')
('Epoch: 580/1000', 'Iteration: 5225', 'Train loss: 0.323988', 'Train acc: 0.915000')
('Epoch: 581/1000', 'Iteration: 5230', 'Train loss: 0.189033', 'Train acc: 0.948333')
('Epoch: 581/1000', 'Iteration: 5235', 'Train loss: 0.263845', 'Train acc: 0.921667')
('Epoch: 582/1000', 'Iteration: 5240', 'Train loss: 0.169135', 'Train acc: 0.945000')
('Epoch: 582/1000', 'Iteration: 5245', 'Train loss: 0.172414', 'Train acc: 0.956667')
('Epoch: 583/1000', 'Iteration: 5250', 'Train loss: 0.279783', 'Train acc: 0.918333')
('Epoch: 583/1000', 'Iteration: 5255', 'Train loss: 0.

KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots() 
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 20
fig_size[1] = 10
plt.rcParams["figure.figsize"] = fig_size
newlist = train_window_cnt
print [val[1] for val in newlist]
ax.set_title(u"Số lượng của sổ mỗi hành động")
ax.set_xlabel(u"Hành động", fontsize=20)
ax.set_ylabel(u"Số ô cửa số", fontsize=20)
plt.bar(range(len(newlist)), list(newlist.values()), align='center')
plt.xticks(range(len(newlist)), list(newlist.keys()), rotation=70, fontsize=20)

# plt.bar(range(len(top20)), [val[1] for val in top20], align='center')
# plt.xticks(range(len(top20)), [val[0] for val in top20])
# plt.xticks(rotation=70)
# plt.show()

In [ ]:
x = list(train_window_cnt.keys())
y = list(train_window_cnt.values())
plt.bar(x, y)
for a,b in zip(x, y):
    plt.text(a, b, str(b))
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
y_true = np.array(['cat', 'dog', 'pig', 'cat', 'dog', 'pig'])
y_pred = np.array(['cat', 'pig', 'dog', 'cat', 'cat', 'dog'])
print confusion_matrix(y_true, y_pred)
precision_recall_fscore_support(y_true, y_pred, average=None,  labels=['pig', 'dog', 'cat'])

In [ ]:
listone = [1,2,3]
listtwo = [4,5,6]
mergedlist = []
mergedlist.extend(listone)
mergedlist.extend(listtwo)
print mergedlist


In [ ]:
def getWindow(dir_path, target_path, is_training=True):
    num_anno = len(annotation)
    gears2_data = [[] for anno in annotation];
    wax3_data = [[] for anno in annotation];
    start_annotation = 0
    wax3_file = dir_path + '/' + WAX3_FILE
    gears2_file = dir_path + '/' + GEARS2_FILE
    with open(gears2_file, 'r') as gears2_csv, open(wax3_file, 'r') as wax3_csv:
        gears2_csv_reader = csv.reader(gears2_csv, delimiter=',')
        wax3_csv_reader = csv.reader(wax3_csv, delimiter=',')
        for row in gears2_csv_reader:
            ts = getMilisecond(row[0].strip())
            for i in range(num_anno):
                if ts >= time_range[i]['start'] and ts < time_range[i]['end']:
                    x_acc = float(row[1].strip())
                    y_acc = float(row[2].strip())
                    z_acc = float(row[3].strip())
                    x_gyr = float(row[4].strip())
                    y_gyr = float(row[5].strip())
                    z_gyr = float(row[6].strip())
                    gears2_data[i].append([ts, x_acc, y_acc, z_acc, x_gyr, y_gyr, z_gyr])
                    
        for row in wax3_csv_reader:
            ts = getMilisecond(row[0].strip())
            for i in range(num_anno):
                if ts >= time_range[i]['start'] and ts < time_range[i]['end']:
                    x_acc = float(row[1].strip())
                    y_acc = float(row[2].strip())
                    z_acc = float(row[3].strip())
                    wax3_data[i].append([ts, x_acc, y_acc, z_acc])
    gears2_data = sorted(gears2_data, key=itemgetter(0))
    wax3_file = sorted(wax3_file, key=itemgetter(0))
    
    for i in range(num_anno):
        num_windows = getTotalWindowSize(len(gears2_data[i]))
        startWindow = 0
#         print num_windows
        for j in range(num_windows):
            windowSliced = gears2_data[i][startWindow:startWindow+WINDOWN_SIZE]
            if(len(windowSliced) < WINDOWN_SIZE):
                break
            startWindow += WINDOWN_SIZE
            startTs = windowSliced[0][0]
            endTs = windowSliced[WINDOWN_SIZE - 1][0]
            equivWax3Data = []
            # get data from wax3 and group with gears2
            for k in range(len(wax3_data[i])):
                wax3Ts = wax3_data[i][k][0]
                if(wax3Ts > endTs):
                    break
                if wax3Ts >= startTs and wax3Ts <= endTs:
                    equivWax3Data.append(wax3_data[i][k])
            zero_arr = [0 for zit in range(WINDOWN_SIZE)]
            while len(equivWax3Data) < WINDOWN_SIZE:
                equivWax3Data.append(zero_arr);
            equivWax3Data = equivWax3Data[0:WINDOWN_SIZE]
#             print (startTs, endTs, len(windowSliced), len(equivWax3Data))
            #export windows to file
            x_watch_acc = y_watch_acc = z_watch_acc = "";
            x_watch_gyr = y_watch_gyr = z_watch_gyr = "";
            x_sensor_acc = y_sensor_acc = z_sensor_acc = "";
#             if len(windowSliced) > 150 or len(equivWax3Data) > 150:
#                 print(i, j, len(windowSliced), len(equivWax3Data))
            for k in range(WINDOWN_SIZE):
                x_watch_acc = x_watch_acc + " " + str(windowSliced[k][1])
                y_watch_acc = y_watch_acc + " " + str(windowSliced[k][2])
                z_watch_acc = z_watch_acc + " " + str(windowSliced[k][3])
                
                x_watch_gyr = x_watch_gyr + " " + str(windowSliced[k][4])
                y_watch_gyr = y_watch_gyr + " " + str(windowSliced[k][5])
                z_watch_gyr = z_watch_gyr + " " + str(windowSliced[k][6])
                
                x_sensor_acc = x_sensor_acc + " " + str(equivWax3Data[k][1])
                y_sensor_acc = y_sensor_acc + " " + str(equivWax3Data[k][2])
                z_sensor_acc = z_sensor_acc + " " + str(equivWax3Data[k][3])
                

            with open(x_watch_acc_file, "a") as fw:
                fw.write(x_watch_acc + "\n")
            with open(y_watch_acc_file, "a") as fw:
                fw.write(y_watch_acc + "\n")
            with open(z_watch_acc_file, "a") as fw:
                fw.write(z_watch_acc + "\n")
                
            with open(x_watch_gyr_file, "a") as fw:
                fw.write(x_watch_gyr + "\n")
            with open(y_watch_gyr_file, "a") as fw:
                fw.write(y_watch_gyr + "\n")
            with open(z_watch_gyr_file, "a") as fw:
                fw.write(z_watch_gyr + "\n")
                
            with open(x_sensor_acc_file, "a") as fw:
                fw.write(x_sensor_acc + "\n")
            with open(y_sensor_acc_file, "a") as fw:
                fw.write(y_sensor_acc + "\n")
            with open(z_sensor_acc_file, "a") as fw:
                fw.write(z_sensor_acc + "\n")
            with open(class_file, "a") as fw:
                fw.write(str(class_label_int[annotation[i]]) + "\n")
        train_window_cnt[class_label_vn[int(class_label_int[annotation[i]])]]+=num_windows
        total_window+=num_windows;
#     return wax3_data
        
# exportData('./datasets/PTIT/001/in/', DATASET_TRAIN)